In [1]:
import os
import weaviate

weaviate_apikey = os.getenv("WEAVIATE_APIKEY")
client = weaviate.Client(
    "http://localhost:8080",
    auth_client_secret=weaviate.auth.AuthApiKey(weaviate_apikey),
)

# How many paragraphs we have in the database?
client.query.aggregate("passage").with_meta_count().do()


{'data': {'Aggregate': {'Passage': [{'meta': {'count': 286}}]}}}

In [19]:
# Ask a sample question
question = "What is Antibacterial Activity?"

results = (
    client.query.get(
        "Passage", ["paper_id", "text_content", "topic", "preprocessor_id"]
    )
    .with_additional(["distance"])
    .with_near_text({"concepts": [question], "distance": 0.7})
    .with_limit(3)
    .do()
)


In [21]:
ys = results["data"]["Get"]["Passage"]


In [22]:
ys


[{'_additional': {'distance': 0.34221327},
  'paper_id': '5a072cc8cf58f17b6189544b',
  'preprocessor_id': 'haystack_v0.0.1',
  'text_content': 'antitumor efficacy by inactivating the VRP [18]. The LSFVIL12 particles injected intraperitoneally in mice implanted subcutaneously with pancreatic cancer cells resulted in reduction of tumor size if vaccinations were initiated either immediately after tumor implantation or up to 3 weeks after implantation [18]. A Phase I human clinical trial in kidney carcinoma and melanoma patients using LSFV-IL12 delivered\n\nintravenously has been described and preliminary data have been provided that indicate this formulation (1 Â 108 or 5 Â 109 particles/m2 body surface) was well tolerated and that a transient increase (five- to tenfold) in IL-12 serum levels was detected within the first 3–4 days after injection [16–18]. A complete summary of the clinical outcomes of this clinical trial has not yet been published. A study protocol for a Phase I/II clinic

In [ ]:
# What are the related paragraphs?
for x in result["data"]["Get"]["Passage"]:
    print(x["text_content"], "\n", "-" * 180)

In [ ]:
raw_vec = result["data"]["Get"]["Passage"][0]["_additional"]["vector"]
raw_vec

## With pre-filtering

In [ ]:
where_filter = {
    "path": ["topic"],
    "operator": "Equal",
    "valueText": "covid-19",
}

# where_filter = {
#     "path": ["preprocessor_id"],
#     "operator": "Equal",
#     "valueText": "haystack-v0.0.1",
# }


result = (
    client.query.get("Passage", ["text_content", "topic", "preprocessor_id"])
    .with_where(where_filter)
    .with_near_text(question)
    .with_limit(10)
    .with_additional(["vector", "distance", "id"])
    .do()
)
